In [ ]:
# Burst Configuration
# Change these parameters for your burst:
burst_name = "freya"      # Options: casey, chromatica, freya, hamilton, isha, johndoeII, mahi, oran, phineas, whitney, wilhelm, zach
telescope = "dsa"         # Options: dsa, chime (chime requires data conversion)
nsubbands = 4             # Number of frequency sub-bands for ACF calculation

# Advanced: Set to True to use equal S/N sub-banding instead of equal bandwidth
use_snr_subbanding = True

In [ ]:
# Matplotlib backend for interactive widgets
%matplotlib widget

In [ ]:
# Imports and Setup
import matplotlib.pyplot as plt
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['STIXGeneral', 'DejaVu Serif'],
    'mathtext.fontset': 'stix',
    'font.size': 18,
    'axes.unicode_minus': True,
})
plt.ioff()  # Display figures manually as widgets

import sys, json, pickle, logging
import numpy as np
from pathlib import Path

# Add scint_analysis to path
pkg_root = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
sys.path.insert(0, str(pkg_root))

# Import pipeline modules
from scint_analysis import config, pipeline, plotting, analysis, widgets
from scint_analysis.core import ACF

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Define paths
BURST_CONFIG_PATH = pkg_root / f'configs/bursts/{burst_name}_{telescope}.yaml'
ACF_RESULTS_PATH = pkg_root / f'data/cache/{burst_name}/{burst_name}_acf_results.pkl'

print(f"Analyzing: {burst_name} ({telescope.upper()})")
print(f"Config: {BURST_CONFIG_PATH}")
print(f"ACF cache: {ACF_RESULTS_PATH}")

## Step 1: Update Configuration

Update the number of sub-bands in the configuration file.

In [ ]:
# Update configuration
key_to_update = ['analysis', 'acf', 'num_subbands']
config.update_yaml_config(BURST_CONFIG_PATH, key_to_update, nsubbands)

if use_snr_subbanding:
    key_to_update = ['analysis', 'acf', 'use_snr_subbanding']
    config.update_yaml_config(BURST_CONFIG_PATH, key_to_update, True)

print(f"Configuration updated: {nsubbands} sub-bands")

## Step 2: Load Data and Initialize Pipeline

Load the burst configuration and prepare the data for analysis.

In [ ]:
# Load configuration
analysis_config = config.load_config(BURST_CONFIG_PATH)
print("--- Loaded Configuration ---")
print(json.dumps(analysis_config, indent=2))

# Initialize pipeline
print("\n--- Initializing Scintillation Pipeline ---")
scint_pipeline = pipeline.ScintillationAnalysis(analysis_config)
scint_pipeline.prepare_data()

print(f"Pipeline ready! Spectrum shape: {scint_pipeline.masked_spectrum.power.shape}")
print(f"Frequency range: {scint_pipeline.masked_spectrum.frequencies.min():.1f} - {scint_pipeline.masked_spectrum.frequencies.max():.1f} MHz")
print(f"Time bins: {len(scint_pipeline.masked_spectrum.times)}")

## Step 3: Interactive Window Selection

**Select on-pulse and off-pulse regions** using the interactive widget.

**Instructions:**
1. Adjust the sliders to define on-pulse (burst) and off-pulse (noise) regions
2. Use the zoom slider to focus on the burst
3. Click "Save windows to YAML" to persist your selection

The selected windows will be used to:
- Calculate the burst spectrum (on-pulse)
- Estimate noise properties (off-pulse)
- Properly normalize the ACF

In [ ]:
# Launch interactive window selector
if scint_pipeline.masked_spectrum is None:
    print("Error: Please run the pipeline initialization first.")
else:
    widgets.interactive_window_selector(scint_pipeline, BURST_CONFIG_PATH)

In [ ]:
# Switch to inline plotting for ACF calculation
plt.close('all')
%matplotlib inline

## Step 4: Run ACF Calculation

Calculate the frequency auto-correlation functions for each sub-band.

**What happens:**
- Burst spectrum is split into sub-bands (equal bandwidth or equal S/N)
- ACF is computed for each sub-band
- Noise bias is removed via Monte Carlo template subtraction
- Statistical and systematic errors are estimated
- Results are cached for fitting

In [ ]:
# Run the pipeline
scint_pipeline.run()
print("ACF calculation complete!")

# Save ACF results for fitting
if scint_pipeline.acf_results:
    ACF_RESULTS_PATH.parent.mkdir(parents=True, exist_ok=True)
    with open(ACF_RESULTS_PATH, "wb") as f:
        pickle.dump(scint_pipeline.acf_results, f)
    print(f"ACF results saved to {ACF_RESULTS_PATH}")
    
    # Display summary
    print("\n--- ACF Summary ---")
    for i, freq in enumerate(scint_pipeline.acf_results['subband_center_freqs_mhz']):
        bw = scint_pipeline.acf_results['subband_num_channels'][i] * scint_pipeline.acf_results['subband_channel_widths_mhz'][i]
        print(f"Sub-band {i}: {freq:.1f} MHz (BW: {bw:.1f} MHz)")

In [ ]:
# Switch back to widget backend for interactive fitting
%matplotlib widget

## Step 5: Interactive ACF Fitting

**Fit composite models** to the ACFs using the interactive dashboard.

**Features:**
- Select 1-3 model components: Lorentzian, Gaussian, Gen-Lorentz, Power-law
- Adjust initial parameter guesses with sliders
- Live plot preview (flicker-free)
- Fit with lmfit using least-squares
- Save fit results to YAML configuration

**Model Interpretation:**
- **Lorentzian**: Standard thin-screen diffractive scattering (γ = scintillation bandwidth)
- **Gaussian**: Pulse self-noise or resolved scattering
- **Gen-Lorentz**: Power-law tails with index α (Kolmogorov → α = 5/3)
- **Power-law**: Direct measurement of scattering tail behavior

**Instructions:**
1. Select sub-band from dropdown
2. Choose model components (1-3)
3. Adjust parameters with sliders
4. Click "Fit Model" to run least-squares fit
5. Click "Save Fit to YAML" to store results
6. Repeat for all sub-bands and models of interest

In [ ]:
# Load cached ACF results
with open(ACF_RESULTS_PATH, 'rb') as f:
    acf_results = pickle.load(f)

# Launch ACF fitter dashboard
widgets.acf_fitter_dashboard(acf_results, BURST_CONFIG_PATH)

In [ ]:
# Switch back to inline plotting for publication figures
plt.close('all')
%matplotlib inline

## Step 6: Generate Publication Plots

Create high-quality 3-panel plots showing:
- **Panel 1**: Full ACF with model fit and component breakdown
- **Panel 2**: Zoomed view near zero lag
- **Panel 3**: Fit residuals

The plot includes:
- Best-fit parameters with compact error notation (e.g., "17.3(15)" means 17.3 ± 1.5)
- Goodness-of-fit statistics (reduced χ², BIC)
- Individual model components
- Fit range indicator

In [ ]:
# Configuration for publication plot
subband_to_plot = 0                        # Sub-band index (0 to nsubbands-1)
model_name_to_plot = "Lorentzian"          # Must match saved fit name exactly
save_path = f"plots/{burst_name}_subband{subband_to_plot}_acf.pdf"

# Load ACF data
lags = acf_results["subband_lags_mhz"][subband_to_plot]
acf_data = acf_results["subband_acfs"][subband_to_plot]
errs_data = acf_results.get("subband_acfs_err", [None]*len(acf_results["subband_acfs"]))[subband_to_plot]
current_acf_obj = ACF(acf_data, lags, acf_err=errs_data)

# Load saved fit from YAML
reconstructed_fit = analysis.load_saved_fit(
    config_path=BURST_CONFIG_PATH,
    subband_index=subband_to_plot,
    model_name=model_name_to_plot,
    lags=current_acf_obj.lags
)

# Generate publication plot
if reconstructed_fit:
    plotting.plot_publication_acf(
        acf_obj=current_acf_obj,
        best_fit_curve=reconstructed_fit["best_fit_curve"],
        component_curves=reconstructed_fit["component_curves"],
        redchi=reconstructed_fit["redchi"],
        bic=reconstructed_fit["bic"],
        params=reconstructed_fit["params"],
        fit_range_mhz=reconstructed_fit["fit_range_mhz"],
        save_path=save_path
    )
    print(f"Publication plot saved to: {save_path}")
else:
    print(f"Could not load fit '{model_name_to_plot}' for sub-band {subband_to_plot}")
    print("Make sure you've saved the fit using the dashboard first.")

## Step 7: Multi-Subband Analysis (Optional)

Generate plots for all sub-bands at once.

In [ ]:
# Plot all sub-bands with the same model
model_for_all = "Lorentzian"  # Change to your fitted model

for i in range(len(acf_results["subband_acfs"])):
    lags = acf_results["subband_lags_mhz"][i]
    acf_data = acf_results["subband_acfs"][i]
    errs_data = acf_results.get("subband_acfs_err", [None]*len(acf_results["subband_acfs"]))[i]
    acf_obj = ACF(acf_data, lags, acf_err=errs_data)
    
    fit_data = analysis.load_saved_fit(
        config_path=BURST_CONFIG_PATH,
        subband_index=i,
        model_name=model_for_all,
        lags=lags
    )
    
    if fit_data:
        freq = acf_results["subband_center_freqs_mhz"][i]
        save_path = f"plots/{burst_name}_subband{i}_{freq:.0f}MHz_acf.pdf"
        
        plotting.plot_publication_acf(
            acf_obj=acf_obj,
            best_fit_curve=fit_data["best_fit_curve"],
            component_curves=fit_data["component_curves"],
            redchi=fit_data["redchi"],
            bic=fit_data["bic"],
            params=fit_data["params"],
            fit_range_mhz=fit_data["fit_range_mhz"],
            save_path=save_path
        )
        print(f"Sub-band {i} ({freq:.0f} MHz): saved to {save_path}")
    else:
        print(f"Sub-band {i}: No '{model_for_all}' fit found, skipping.")

## Summary

### Analysis Complete!

**What you've done:**
1. Loaded dynamic spectrum for `{burst_name}` from `{telescope.upper()}`
2. Selected on/off-pulse windows
3. Calculated ACFs for {nsubbands} sub-bands
4. Fit scintillation models to measure bandwidth (γ) and modulation (m)
5. Generated publication-quality plots

### Next Steps

**Physical Interpretation:**
- Extract γ(ν) scaling → power-law index α
- Compare α to theoretical predictions (thin-screen: α ≈ 4, Kolmogorov: α ≈ 11/3)
- Estimate screen distance using Fresnel scale
- Compare CHIME + DSA-110 measurements (if available)

**Additional Analyses:**
- Intra-pulse scintillation evolution (if enabled in config)
- Multi-component model comparison (AIC/BIC)
- Cross-correlation with scattering measurements
- Host galaxy screen distance estimates

### Refactored Architecture Benefits

This workflow uses:
- `widgets.interactive_window_selector()` - Was 110 lines, now 1 call
- `widgets.acf_fitter_dashboard()` - Was 363 lines, now 1 call  
- `plotting.plot_publication_acf()` - Was 250 lines, now 1 call

**Result:** 98% reduction in notebook complexity while maintaining full functionality!